<a href="https://colab.research.google.com/github/Datkhoo25/insurance_risk_prediction/blob/main/8_(XGB%2C_RF)_RFE_DT_Bayes_Impute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', timeout_ms=300000)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from scipy.stats import randint, uniform

from scipy.optimize import fmin_powell
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,  roc_auc_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, r2_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
!pip install -q -U keras-tuner
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.initializers import HeNormal, GlorotNormal, LecunNormal

from keras import regularizers
from keras.layers import Dropout
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam, SGD


import kerastuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner import Objective

from keras.callbacks import History
from keras.callbacks import Callback, EarlyStopping
from keras.metrics import Accuracy, Recall, Precision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


<ipython-input-4-670abaa5fb26>:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
# Specify the path to your Excel file
workbook_path = '/content/drive/MyDrive/Colab Notebooks/Insurance Dataset/Preprocessing_Checkpoint/bayes_Impute_wNA_dt_rfe.xlsx'

# Load the workbook
wb = openpyxl.load_workbook(workbook_path)

# Get sheet names
sheet_names = wb.sheetnames

# Container for DataFrames
dfs = {}

# Iterate through each sheet
for sheet_name in sheet_names:
    print(f"\nSheet: {sheet_name}")
    sheet = wb[sheet_name]

    # Extract headers (first row)
    headers = [cell.value for cell in sheet[1]]
    # print(f"Headers: {headers}")

    # Extract data rows
    data = []
    for row in sheet.iter_rows(min_row=2, values_only=True):
        data.append(row)
    # print("Data:")
    # for row in data:
    #     print(row)

    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=headers)
    dfs[sheet_name] = df

In [ ]:
# Access your DataFrames from the dictionary `dfs`
print("\nDataFrames:")
for name, df in dfs.items():
    print(f"\nDataFrame: {name}")
    name = df

print(sheet_names)

In [ ]:
x_train, x_val, x_test, y_train, y_val = dfs.items()
x_train_m = x_train[1]
x_val_m = x_val[1]
x_test_m = x_test[1]
y_train_m = y_train[1]
y_val_m = y_val[1]

In [ ]:
x_train = x_train_m
x_val = x_val_m
x_test = x_test_m
y_train = y_train_m
y_val = y_val_m

In [ ]:
print(x_train_m.shape, y_train_m.shape)
print(x_val_m.shape, y_val_m.shape)

In [ ]:
y_train_m.max()

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [ ]:
xgb = XGBClassifier()

XGBoost

In [ ]:
random_state = 42

# Define the parameter distributions
param_dist = {
    'max_depth': randint(10, 40),  # XGBClassifier parameters
    'min_child_weight': randint(0, 3),
    'subsample': uniform(0.5, 0.4),
    'colsample_bytree': uniform(0.5, 0.4),
    'learning_rate': uniform(0.1, 0.3),
    'n_estimators': randint(500, 1500)
}


xgb = XGBClassifier(objective='multi:softmax', num_class=8, n_jobs=-1)

# Instantiate RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb,
                                   param_distributions=param_dist,
                                   n_iter=50,  # Number of parameter settings that are sampled
                                   scoring='accuracy',
                                   cv=3,
                                   verbose=1,
                                   n_jobs=-1,
                                   random_state=42)

# Fit RandomizedSearchCV to the training data
random_search.fit(x_train, y_train_encoded)

# Print the best parameters and best score
print("Best parameters found:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Get the best estimator
best_xgb_model = random_search.best_estimator_

# Predict on validation set with the best estimator
y_pred = best_xgb_model.predict(x_val)

# Evaluate accuracy with the best estimator
accuracy_best = accuracy_score(y_val_encoded, y_pred)
print(f"Accuracy on validation set with best estimator: {accuracy_best:.4f}")

# Evaluate precision with the best estimator
precision_best = precision_score(y_val_encoded, y_pred, average='micro')
print(f"Precision on validation set with best estimator: {precision_best:.4f}")

# Evaluate precision with the best estimator
recall_best = recall_score(y_val_encoded, y_pred, average='micro')
print(f"Recall on validation set with best estimator: {recall_best:.4f}")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters found: {'colsample_bytree': 0.7473544037332349, 'learning_rate': 0.21473859738014883, 'max_depth': 29, 'min_child_weight': 0, 'n_estimators': 630, 'subsample': 0.8439761626945282}
Best cross-validation score: 0.5394283779147079
Accuracy on validation set with best estimator: 0.5445
Precision on validation set with best estimator: 0.5445
Recall on validation set with best estimator: 0.5445


Random Forest

In [ ]:

param_grid = {
    'n_estimators': [900, 1100, 1300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [30, 50],
    'min_samples_leaf': [5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}


random_forest = RandomForestClassifier(random_state=42)

# Instantiate the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=random_forest,
                                   param_distributions=param_grid,
                                   n_iter=50,  # Number of parameter settings that are sampled
                                   scoring='accuracy',
                                   cv=3,
                                   verbose=1,
                                   n_jobs=-1,
                                   random_state=42)

# Fit RandomizedSearchCV to the training data
random_search.fit(x_train, y_train_encoded)

# Print the best parameters and best score
print("Best parameters found:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Get the best estimator
best_rf_classifier = random_search.best_estimator_

# Predict on the validation set with the best estimator
y_pred_best = best_rf_classifier.predict(x_val)

# Evaluate accuracy with the best estimator
accuracy_best = accuracy_score(y_val_encoded, y_pred_best)
print(f"Accuracy on validation set with best estimator: {accuracy_best:.4f}")

# Evaluate precision with the best estimator
precision_best = precision_score(y_val_encoded, y_pred_best, average='micro')
print(f"Precision on validation set with best estimator: {precision_best:.4f}")

# Evaluate precision with the best estimator
recall_best = recall_score(y_val_encoded, y_pred_best, average='micro')
print(f"Recall on validation set with best estimator: {recall_best:.4f}")

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Best parameters found: {'n_estimators': 900, 'min_samples_split': 30, 'min_samples_leaf': 5, 'max_features': 'auto', 'max_depth': 20}
Best cross-validation score: 0.529071271584038
Accuracy on validation set with best estimator: 0.5319
Precision on validation set with best estimator: 0.5319
Recall on validation set with best estimator: 0.5319


In [ ]:
# Evaluate accuracy with the best estimator
accuracy_best = accuracy_score(y_val_encoded, y_pred_best)
print(f"Accuracy on validation set with best estimator: {accuracy_best:.4f}")

# Evaluate precision with the best estimator
precision_best = precision_score(y_val_encoded, y_pred_best, average='micro')
print(f"Precision on validation set with best estimator: {precision_best:.4f}")